In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

In [2]:
columns_drop = ['artist_id', 'artist_mbid',
       'artist_playmeid', 'artist_7digitalid',
       'release_7digitalid', 'song_id', 'title',
       'track_7digitalid', 'similar_artists', 'artist_terms',
       'artist_terms_freq', 'artist_terms_weight', 'analysis_sample_rate',
       'audio_md5', 'segments_start',
       'segments_confidence', 'segments_pitches', 'segments_timbre',
       'segments_loudness_max', 'segments_loudness_max_time',
       'segments_loudness_start', 'sections_start', 'sections_confidence',
       'beats_start', 'beats_confidence', 'bars_start', 'bars_confidence',
       'tatums_start', 'tatums_confidence', 'artist_mbtags',
       'artist_mbtags_count', 'analysis_url',
        'id', 'track_href', 'type', 'uri']

columns_desired = ['artist_familiarity', 'artist_hotttnesss', 'artist_playmeid', 'artist_7digitalid', 'artist_latitude',
       'artist_longitude', 'artist_location', 'artist_name', 'release', 'song_hotttnesss',
     'danceability', 'duration', 'end_of_fade_in', 'energy',
       'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence',
       'start_of_fade_out', 'tempo', 'time_signature',
       'time_signature_confidence', 'year', 'acousticness', 
       'danceability.1', 'duration_ms', 'energy.1', 'instrumentalness',
       'key.1', 'liveness', 'loudness.1', 'mode.1', 'speechiness', 'tempo.1',
       'time_signature.1', 'valence']

numeric_columns = ['artist_familiarity', 'artist_hotttnesss', 'artist_playmeid', 'artist_7digitalid','artist_latitude', 'artist_longitude', 
        'song_hotttnesss', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'key', 'key_confidence', 
        'loudness', 'mode', 'mode_confidence',
        'start_of_fade_out', 'tempo', 'time_signature',
        'time_signature_confidence', 'year', 'acousticness', 
        'danceability.1', 'duration_ms', 'energy.1', 'instrumentalness',
        'key.1', 'liveness', 'loudness.1', 'mode.1', 'speechiness', 'tempo.1',
        'time_signature.1', 'valence']

In [3]:
million_csv = pd.read_csv("FeatureEngineered_genre.csv")

In [4]:
# Filter out the columns I want
million_data = million_csv
len(million_data.columns)

45

In [5]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

from keras.models import Sequential

from sklearn.preprocessing import LabelEncoder
from keras.optimizers import SGD

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
random_state = 100
np.random.seed(random_state)
train, test = train_test_split(million_data, test_size=.2, random_state = random_state)

In [7]:
y_train = train.iloc[:,train.columns == 'year']
X_train = train.loc[:, train.columns != 'year']
y_test = test.iloc[:,train.columns == 'year']
X_test = test.loc[:, test.columns != 'year']
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
print(y_train.min(axis=0))
print(y_test.min(axis=0))

print(y_train.max(axis=0))
print(y_test.max(axis=0))

year    1910
dtype: int32
year    1920
dtype: int32
year    2010
dtype: int32
year    2010
dtype: int32


In [8]:
def create_model(learning_rate = .001, momentum = .5, init_mode = 'zero', activation = 'softplus'):
    # From medium Building Neural Networks using Keras for Classification article
    model = Sequential()

    
    model.add(Dense(32, activation=activation, kernel_initializer=init_mode))

    model.add(Dense(16, activation=activation, kernel_initializer=init_mode))

    model.add(Dense(8, activation=activation, kernel_initializer=init_mode))

    model.add(Dense(4, activation=activation, kernel_initializer=init_mode))
    
    model.add(Dense(4, activation=activation, kernel_initializer=init_mode))

    #Output Layer
    model.add(Dense(1, kernel_initializer=init_mode))
    
#     sgd = SGD(lr = learning_rate, momentum=momentum)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics =['accuracy'])
    
    return model

model = KerasRegressor(build_fn=create_model, verbose = 0)

In [11]:
dropout_rate = np.arange(0, .6, .2)
epochs=[10]
batch_size = [1000]
# weight_constraint = [1, 2, 3, 4, 5]
momentum = np.arange(.1, .6, .1)
learning_rate = np.arange(.03, .05,  .005)
# Help from machinelearningmastery  article on How to grid search... by Jason Brownlee for these parameters
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'linear']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero']

# optimizer = ['adadelta']
# activation = ['softmax']
# init_mode=['uniform']
param_grid = dict(#optimizer=optimizer, 
#                   dropout_rate=dropout_rate, 
                  epochs=epochs, 
                  momentum=momentum,
                  batch_size=batch_size, 
#                   weight_constraint=weight_constraint, 
                  learning_rate=learning_rate,
                  activation = activation,
                  init_mode = init_mode
                 )
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv = 3, n_jobs=6, scoring='neg_mean_squared_error', verbose=1)
grid_result = grid.fit(X_train.values, y_train.values)

Fitting 3 folds for each of 600 candidates, totalling 1800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    9.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  6.3min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed: 20.8min
exception calling callback for <Future at 0x7f4dfd5cb898 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/brandon/python3_environments/ds19_project/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/brandon/python3_environments/ds19_project/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "/home/brandon/python3_environments/ds19_project/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_b

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
print(grid_result.best_params_)

In [13]:
model = create_model()
model.fit(X_train.values, y_train.values, epochs=100, batch_size=40)
output = model.predict(X_test.values)
print(output.max())
print(output.min())

Epoch 1/100
7164/7164 [==============================] - 0s 69us/step - loss: 1480957.2203 - acc: 5.5835e-04
Epoch 2/100
7164/7164 [==============================] - 0s 28us/step - loss: 305340.7060 - acc: 8.3752e-04
Epoch 3/100
7164/7164 [==============================] - 0s 29us/step - loss: 22504.3161 - acc: 0.0121
Epoch 4/100
7164/7164 [==============================] - 0s 28us/step - loss: 463.8593 - acc: 0.0195
Epoch 5/100
7164/7164 [==============================] - 0s 29us/step - loss: 428.3863 - acc: 0.0225
Epoch 6/100
7164/7164 [==============================] - 0s 27us/step - loss: 402.7036 - acc: 0.0186
Epoch 7/100
7164/7164 [==============================] - 0s 28us/step - loss: 368.6720 - acc: 0.0218
Epoch 8/100
7164/7164 [==============================] - 0s 27us/step - loss: 330.3902 - acc: 0.0222
Epoch 9/100
7164/7164 [==============================] - 0s 31us/step - loss: 289.1745 - acc: 0.0255
Epoch 10/100
7164/7164 [==============================] - 0s 29us/step - l

In [371]:
# One hot encoding
encoder = LabelEncoder()
encoder.fit(y_train_array)
encoded_Y = encoder.transform(y_train_array)
dummy_y_train = np_utils.to_categorical(encoded_Y)

encoded_Y = encoder.transform(y_test_array)
dummy_y_test = np_utils.to_categorical(encoded_Y)

In [170]:
score = model.evaluate(X_test.values, y_test.values)
print(score)
print(model.metrics_names)

1791/1791 [==============================] - 0s 25us/step
[52201.31471594082, 0.0005583472920156337]
['loss', 'acc']
